In [4]:
import numpy as np
from Robot import UR5Arm
from fwdkin import fwdkin
import math
from path_planning import path_planner
from Arm_Lib import Arm_Device
from utils import rot
Arm = Arm_Device()


In [5]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 

P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = None 
dofbot = UR5Arm(P, H, limits)

[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]


In [6]:
Kp = 0.0000001
Ki = 0.00000001
Kd = 0.00000001

doPID = True 

time = 800
N = 5

hover = [90, 40, 40, 90, 90]
hover_right = [65, 40, 40, 90, 90]
hover_2 = [62, 61, -13, 100, 90]



In [21]:
home = np.array([90, 90, 90, 90, 90])[None].T*math.pi/180

start = np.array( [90, 110, 50, 10, 90])[None].T*math.pi/180
red = np.array(   [115, 28, 45, 75, 90])[None].T*math.pi/180
yellow = np.array([62, 31, 31, 90, 90])[None].T*math.pi/180
blue = np.array(  [44, 66, 20, 28, 90])[None].T*math.pi/180
green = np.array([136, 66, 20, 28, 90])[None].T*math.pi/180


block4 = np.array([90, 72, 49, 13, 90])[None].T*math.pi/180


R_start, P_start = fwdkin(dofbot, start)
R_red, P_red = fwdkin(dofbot, red)
R_yellow, P_yellow = fwdkin(dofbot, yellow)
R_blue, P_blue = fwdkin(dofbot, blue)
R_green, P_green = fwdkin(dofbot, green)
# P_blue = P_yellow - np.array([0,0.14,0])
# R_blue = rot(ex, 40*math.pi/180) @ rot(ez, -45*math.pi/180) @ R_mid
R_home, P_home = fwdkin(dofbot, home)
R_b4, P_b4 = fwdkin(dofbot, block4)


R_mid_5 = rot(ex, -115*math.pi/180) @ R_home
R_mid = rot(ex, -140*math.pi/180) @ R_home
R_mid_4 = rot(ex, -20*math.pi/180) @ R_mid_5

#          x    y    z 
P_mid =   [0, 0.2, 0.062]
P_mid_2 = [0, 0.195, 0.088]
P_mid_3 = [0, 0.183, 0.12]
P_mid_4 = [0, 0.18, 0.158]
P_mid_5 = [0, 0.17, 0.188]


print(R_red, R_yellow)
print(P_red, P_yellow)


path_planner(Arm, dofbot, N, R_home, P_home, R_mid_5, P_mid_5, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_red, P_red + np.array([0, 0, 0.05]), Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_red, P_red + np.array([0, 0, 0.05]), R_red, P_red, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_red, P_red, R_mid, P_mid, Kp, Ki, Kd, 150, time, doPID)
path_planner(Arm, dofbot, N, R_mid, P_mid, R_mid_5, P_mid_5, Kp, Ki, Kd, 90, time, doPID)


path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_yellow, P_yellow + np.array([0, 0, 0.05]), Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_yellow, P_yellow + np.array([0, 0, 0.05]), R_yellow, P_yellow, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_yellow, P_yellow, R_mid_5, P_mid_5, Kp, Ki, Kd, 150, time, doPID)
path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_mid, P_mid_2, Kp, Ki, Kd, 150, time, doPID)

path_planner(Arm, dofbot, N, R_mid, P_mid_2, R_mid_5, P_mid_5, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_blue, P_blue, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_blue, P_blue, R_mid_5, P_mid_5, Kp, Ki, Kd, 150, time, doPID)
path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_mid, P_mid_3, Kp, Ki, Kd, 150, time, doPID)


path_planner(Arm, dofbot, N, R_mid, P_mid_3, R_mid_5, P_mid_5, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_green, P_green, Kp, Ki, Kd, 90, time, doPID)
path_planner(Arm, dofbot, N, R_green, P_green, R_mid_5, P_mid_5, Kp, Ki, Kd, 150, time, doPID)
path_planner(Arm, dofbot, N, R_mid_5, P_mid_5, R_b4, P_b4, Kp, Ki, Kd, 150, time, doPID)

path_planner(Arm, dofbot, N, R_b4, P_b4, R_mid_5, P_mid_5, Kp, Ki, Kd, 90, time, doPID)


print("passed Path Following")

[[ 3.58400612e-01 -2.23953558e-01 -9.06307787e-01]
 [-7.68592593e-01  4.80269956e-01 -4.22618262e-01]
 [ 5.29919264e-01  8.48048096e-01 -9.41522522e-17]] [[-4.14518786e-01  2.20403548e-01 -8.82947593e-01]
 [-7.79596452e-01  4.14518786e-01  4.69471563e-01]
 [ 4.69471563e-01  8.82947593e-01 -9.80268747e-17]]
[-0.11042525  0.23680771  0.05112049] [0.12091552 0.22740902 0.04798565]
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-5.93641222e-08  1.31662043e-08 -3.82107213e-09  2.14878152e-05
  9.68848149e-05 -7.24935068e-07]
Inverse Kinematics Converged after 166 iterations
q_0: [1.35254194 1.57309424 1.56500276 1.57429191 1.78905071]
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-2.12989104e-05 -2.41208330e-05 -2.07527463e-05  5.43824948e-06
 -6.05502547e-05  9.44446327e-05]
Inv

# 